In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the CSV file into a DataFrame
df = pd.read_csv(r"C:\Users\karrt\Documents\ProjectGRO\Diff_proj\movies.csv")
# Preprocess the genres column
df['genres'] = df['genres'].str.replace('|', ' ')

df.head()




C:\Users\karrt\AppData\Local\Temp\ipykernel_12840\1770602479.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['genres'] = df['genres'].str.replace('|', ' ')


,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')
print(tfidf)

# Fit and transform the genres column
tfidf_matrix = tfidf.fit_transform(df['genres'])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)



# # Initialize TF-IDF Vectorizer
# tfidf = TfidfVectorizer(stop_words='english')

# # Fit and transform the genres column
# tfidf_matrix = tfidf.fit_transform(df['genres'])

# # Compute cosine similarity matrix
# cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

TfidfVectorizer(stop_words='english')


In [ ]:
print(cosine_sim.shape)

(62423, 62423)


In [ ]:
print(tfidf_matrix.shape)

(62423, 23)


In [ ]:
# Get feature names
feature_names = tfidf.get_feature_names_out()

# Print the feature names
print(feature_names)

['action' 'adventure' 'animation' 'children' 'comedy' 'crime'
 'documentary' 'drama' 'fantasy' 'fi' 'film' 'genres' 'horror' 'imax'
 'listed' 'musical' 'mystery' 'noir' 'romance' 'sci' 'thriller' 'war'
 'western']


In [ ]:
# Count the number of movies with genre as "(no genre listed)"
no_genre_movies_count = df[df['genres'] == "(no genres listed)"].shape[0]
print("Number of movies with genre as '(no genre listed)':", no_genre_movies_count)

Number of movies with genre as '(no genre listed)': 5062


In [ ]:
# Remove rows where genre is "(no genres listed)"
df = df[df['genres'] != "(no genres listed)"]

# Verify the removal by printing the shape of the cleaned DataFrame
print("Shape of cleaned DataFrame:", df.shape)


Shape of cleaned DataFrame: (57361, 3)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

glove_embeddings = load_glove_embeddings(r"C:\Users\karrt\Documents\ProjectGRO\Diff_proj\glove.6B.50d.txt")  # Path to GloVe file

In [ ]:

# Find most similar genres to given genres using GloVe embeddings
def find_most_similar_genres(input_genres, embeddings, df, top_n=3):
    input_vectors = []
    for input_genre in input_genres:
        input_vector = np.zeros_like(next(iter(embeddings.values())))
        for word in input_genre.split():
            input_vector += embeddings.get(word.lower(), np.zeros_like(input_vector))
        input_vector /= len(input_genre.split())  # Average the embeddings
        input_vectors.append(input_vector)
    
    # Compute cosine similarity between input genre embeddings and genre embeddings in the dataset
    similarities = []
    for genre in df['genres']:
        genre_vector = np.zeros_like(next(iter(embeddings.values())))
        for word in genre.split():
            genre_vector += embeddings.get(word.lower(), np.zeros_like(genre_vector))
        genre_vector /= len(genre.split())  # Average the embeddings
        similarity = np.mean([cosine_similarity([input_vector], [genre_vector])[0][0] for input_vector in input_vectors])
        similarities.append(similarity)
    
    # Find indices of top N genres with highest similarity
    top_indices = np.argsort(similarities)[-top_n:][::-1]
    
    return [df.iloc[idx]['genres'] for idx in top_indices]


In [ ]:

# Recommend movies based on the closest genres
def recommend_movies_based_on_genres(closest_genres, df, tfidf_vectorizer, cosine_sim, top_n=3):
    # Combine closest genres into a single string
    combined_genres = ' '.join(closest_genres)
    # Transform combined genres into TF-IDF vector
    input_tfidf = tfidf_vectorizer.transform([combined_genres])
    # Compute cosine similarity between combined genres and all movies
    cosine_scores = cosine_similarity(input_tfidf, tfidf_matrix).flatten()
    # Get indices of top n movies with highest cosine similarity
    top_indices = cosine_scores.argsort()[-top_n:][::-1]
    # Retrieve movie titles based on top indices
    recommended_movies = df.iloc[top_indices]['title']
    
    return recommended_movies


In [ ]:
# Example usage
input_genres = input("Enter genres you would like to watch (separated by commas): ")

most_similar_genres = find_most_similar_genres(input_genres.split(','), glove_embeddings, df)
print("Most similar genres to", input_genres, ":", most_similar_genres)

recommended_movies = recommend_movies_based_on_genres(most_similar_genres, df, tfidf, cosine_sim)
print("Recommended Movies:")
print(recommended_movies)

Most similar genres to physical, brutal, madness : ['Action Fantasy Horror War', 'Action Horror War', 'Fantasy Horror War']
Recommended Movies:
61657               Adrenochrome (2018)
40084    The Living and the Dead (2007)
25811                   Stranded (2010)
Name: title, dtype: object
